In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

In [13]:
df = pd.read_pickle(r'shared/Motor_Vehicle_Collisions_-_Crashes.pkl')
df

# question 1: which borough has highest number of crashes since 2012

df['CRASH DATE_CRASH TIME'] = pd.to_datetime(df['CRASH DATE_CRASH TIME'])

In [14]:
df_filtered = df[df['CRASH DATE_CRASH TIME'] >= '2012-01-01']
df_filtered

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018240,2023-07-03 18:05:00,NaN,NaN,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4648110,Sedan,Sedan,NaN,NaN,NaN
2018241,2023-07-22 21:39:00,BRONX,10457.0,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,NaN,1.0,...,NaN,NaN,NaN,NaN,4648117,Sedan,NaN,NaN,NaN,NaN
2018242,2023-07-02 17:55:00,MANHATTAN,10006.0,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4648366,Taxi,NaN,NaN,NaN,NaN
2018243,2023-07-22 13:15:00,QUEENS,11433.0,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,NaN,1.0,...,Driver Inattention/Distraction,NaN,NaN,NaN,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN


In [15]:
#group by "BOROUGH" and count the number

borough_counts = df_filtered.groupby('BOROUGH').size()
borough_counts

#print the max
borough_counts.idxmax()

'BROOKLYN'

QUESTION 2: find out which borough has the maximum number of crashes for every 100,000 people

In [31]:

# Count the crashes per borough
borough_crash_counts = df_filtered.groupby('BOROUGH').size()

# Population data
populations = {
    "BRONX": 1446788,
    "BROOKLYN": 2648452,
    "MANHATTAN": 1638281,
    "QUEENS": 2330295,
    "STATEN ISLAND": 487155
}

# Calculate crashes per 100k population for each borough
crashes_per_100k = {}
for borough, count in borough_crash_counts.items():
    crashes_per_100k[borough] = (count / populations[borough]) * 100000

# Get borough with the maximum crashes per 100k
max_borough = max(crashes_per_100k, key=crashes_per_100k.get)
max_crashes_100k = crashes_per_100k[max_borough]

print(f"{max_borough}, with {max_crashes_100k:.5f} crashes per 100k population")


MANHATTAN, with 19121.62810 crashes per 100k population


question 3
Remove all rows where the contributing factor vehicle 1 is not known, leaving only the crashes where only this information is available.

What proportion of accidents are attributable to this cause?

In [16]:
print(df.columns)

Index(['CRASH DATE_CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE',
       'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME',
       'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
       'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
       'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
       'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED',
       'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2',
       'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4',
       'CONTRIBUTING FACTOR VEHICLE 5', 'COLLISION_ID', 'VEHICLE TYPE CODE 1',
       'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4',
       'VEHICLE TYPE CODE 5'],
      dtype='object')


In [17]:
print(df['CONTRIBUTING FACTOR VEHICLE 1'])

0            Aggressive Driving/Road Rage
1                       Pavement Slippery
2                   Following Too Closely
3                             Unspecified
4                                     NaN
                        ...              
2018240                Turning Improperly
2018241                       Unspecified
2018242    Driver Inattention/Distraction
2018243    Driver Inattention/Distraction
2018244                       Unspecified
Name: CONTRIBUTING FACTOR VEHICLE 1, Length: 2018245, dtype: object


In [18]:
#filter out the "Unspecified"
filtered_df = df[df['CONTRIBUTING FACTOR VEHICLE 1']!= "Unspecified"]

In [19]:
#count the occurrences of each contributing factor
factor_counts = filtered_df['CONTRIBUTING FACTOR VEHICLE 1'].value_counts()
factor_counts

CONTRIBUTING FACTOR VEHICLE 1
Driver Inattention/Distraction                           401262
Failure to Yield Right-of-Way                            119166
Following Too Closely                                    107467
Backing Unsafely                                          75042
Other Vehicular                                           62688
Passing or Lane Usage Improper                            55445
Turning Improperly                                        49908
Passing Too Closely                                       49848
Fatigued/Drowsy                                           47343
Unsafe Lane Changing                                      39711
Traffic Control Disregarded                               35210
Driver Inexperience                                       31274
Unsafe Speed                                              26915
Alcohol Involvement                                       21656
Lost Consciousness                                        20374
Reaction t

In [20]:
#identify the leading factor
Main_factor = factor_counts.idxmax()

#calculate the proportion 
total_crashes = len(filtered_df)
proportion = (factor_counts.max()/total_crashes)*100
print(f"leading cause: {Main_factor},proportion: {proportion:.1f}%")

leading cause: Driver Inattention/Distraction,proportion: 30.3%


QUESTION 4
Continuing from the previous question - the top 3 causes of crashes (ignoring 'Unspecified') account for what proportion of total crashes?

In [21]:
#sum the counts of the top 3 causes
top_3_causes= factor_counts.iloc[:3].sum()
top_3_causes

#calculate the proportion 
proportion_top_3 = (top_3_causes/total_crashes)*100

print(f"proportion of crashes caused by the top 3 factors: {proportion_top_3:.1f}%")

proportion of crashes caused by the top 3 factors: 47.4%


QUESTION 5 

Considering the entire data since 2012 (and, if applicable, ignoring missing values), what is the total count of accidents that involved at least one fatality?

In [22]:
#filter out at least one fatality (>0)
fatality_cases = df_filtered[df_filtered['NUMBER OF PERSONS KILLED']>0]
fatality_cases

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
39,2021-07-09 00:43:00,NaN,NaN,40.720535,-73.888850,"(40.720535, -73.88885)",ELIOT AVENUE,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,4456659,Bus,NaN,NaN,NaN,NaN
148,2021-12-12 09:09:00,NaN,NaN,40.840360,-73.918070,"(40.84036, -73.91807)",JEROME AVENUE,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,4487210,Taxi,NaN,NaN,NaN,NaN
591,2021-04-15 15:18:00,BROOKLYN,11209.0,40.620487,-74.029305,"(40.620487, -74.029305)",4 AVENUE,FOREST PLACE,NaN,0.0,...,NaN,NaN,NaN,NaN,4408063,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
605,2021-04-15 22:36:00,NaN,NaN,NaN,NaN,NaN,Trans- Manhattan Expressway,Amsterdam Avenue,NaN,4.0,...,NaN,NaN,NaN,NaN,4407693,Sedan,NaN,NaN,NaN,NaN
1320,2021-04-17 13:31:00,NaN,NaN,40.782463,-73.978830,"(40.782463, -73.97883)",AMSTERDAM AVENUE,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,4408062,E-Bike,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016625,2023-07-01 00:27:00,QUEENS,11372.0,40.753536,-73.886900,"(40.753536, -73.8869)",80 STREET,34 AVENUE,NaN,3.0,...,Unspecified,NaN,NaN,NaN,4643896,Motorcycle,NaN,NaN,NaN,NaN
2016626,2023-07-09 09:25:00,QUEENS,11103.0,40.764730,-73.912110,"(40.76473, -73.91211)",28 AVENUE,42 STREET,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4643897,Station Wagon/Sport Utility Vehicle,Moped,NaN,NaN,NaN
2016774,2023-07-17 22:15:00,BROOKLYN,11236.0,40.632435,-73.888180,"(40.632435, -73.88818)",ROCKAWAY PARKWAY,SKIDMORE AVENUE,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4646703,Sedan,Motorcycle,NaN,NaN,NaN
2018044,2023-07-22 11:17:00,QUEENS,11429.0,40.705220,-73.727880,"(40.70522, -73.72788)",112 AVENUE,CROSS ISLAND PARKWAY,NaN,0.0,...,NaN,NaN,NaN,NaN,4648067,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN


In [23]:
total_fatal_cases = len(fatality_cases)
total_fatal_cases

2804

QUESTION 6 

On average, for every 1000 accidents, how many have resulted in at least one person dead? (hint: divide the count of accidents with 1 or more deaths with the total number of accidents, and multiply by 1000)

In [24]:
total_cases = len(df_filtered)
fatal_per_1000 = (total_fatal_cases/total_cases)*1000
print(f"on average.for every 1000 accidents, {fatal_per_1000:.2f} have resulted in at least one person dead.")

on average.for every 1000 accidents, 1.39 have resulted in at least one person dead.


QUESTION 7 

What proportion of accidents in the data do not have a Borough code?

In [25]:
non_borough_code = df["BOROUGH"].isnull().sum()
non_borough_code

627854

In [26]:
total_accidents = len(df)

#calculate the proportion 
proportion_non_borough_code = (non_borough_code/total_accidents)*100
proportion_non_borough_code

31.10890897784957

QUESTION 8 

The fields 'VEHICLE TYPE CODE 1' and 'VEHICLE TYPE CODE 2' represent the first two vehicles involved in the accident. Which combination of vehicles have the most number of accidents?

In [27]:
#create and combine two vehicles

df['COMBINED_VEHICLE_TYPE'] = df['VEHICLE TYPE CODE 1'] + ' and ' + df['VEHICLE TYPE CODE 2']

#count for each combo 
combo_counts = df['COMBINED_VEHICLE_TYPE'].value_counts()

#which is the most common one?
most_common_combo = combo_counts.idxmax()

print(f'The most common combination of vehicles is {most_common_combo}')

The most common combination of vehicles is Sedan and Sedan


SyntaxError: invalid syntax (1386261081.py, line 1)